# Data reconciliation

This notebook is used to perform reconciliation on data obtained in realtime with batch data downloaded at once, i.e. once a day. 
As displayed below, the notebook assumes environment variables for the reconciliation parameters. The intended use 
is via invoke target `dev_scripts.lib_tasks_reconcile.run_data_reconciliation_notebook`

## Imports and logging

In [ ]:
import os
import argparse
import logging
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import im_v2.ccxt.data.extract.compare_realtime_and_historical as imvcdecrah

### Logging

In [ ]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

## Reconciliation parameters

To assist debugging you can override any of the parameters after its loaded and rerun reconciler

In [ ]:
os.environ.items()

In [ ]:
# Load relevant environment variables.
config = filter(lambda x: x[0].startswith("DATA_RECONCILE_"), os.environ.items())
# Transform parameter names to the naming conventions used by the reconciler.
config = list(map(lambda x: (x[0].replace("DATA_RECONCILE_", "").lower(), x[1]), config))
# Dict can be passed as a namespace argument.
config = { it[0] : it[1] for it in config }
config

In [ ]:
# Bid_ask_accuracy needs special treatment
#  since int is expected in the reconciler.
config['bid_ask_accuracy'] = None if config['bid_ask_accuracy'] == 'None' else int(config['bid_ask_accuracy'])
# Transform resample_mode to parameter supported
#  by the reconciler
config["resample_1sec"] = config["resample_mode"] == "resample_1sec"
config["resample_1min"] = config["resample_mode"] == "resample_1min"
config

## Initialize Reconciler

In [ ]:
# The class was originally intended to be used via a cmdline script
args = argparse.Namespace(**config)
reconciler = imvcdecrah.RealTimeHistoricalReconciler(args)

In [ ]:
# CCXT Realtime data
reconciler.ccxt_rt.head()

In [ ]:
# Historical data
reconciler.daily_data.head()

## Run reconciliation

In [ ]:
reconciler.run()